In [28]:
import requests
import xml.etree.ElementTree as ET
from datetime import datetime

In [29]:
# API URL for getting current trains
CURRENT_TRAINS_URL = "http://api.irishrail.ie/realtime/realtime.asmx/getCurrentTrainsXML"

def get_train_codes():
    response = requests.get(CURRENT_TRAINS_URL)
    
    if response.status_code != 200:
        print("Failed to fetch current trains")
        return []

    root = ET.fromstring(response.text)
    namespace = {"ns": "http://api.irishrail.ie/realtime/"}
    train_codes = []

    for train in root.findall("ns:objTrainPositions", namespace):
        train_code = train.find("ns:TrainCode", namespace).text
        train_date_raw = train.find("ns:TrainDate", namespace).text  # Example: '31 Mar 2025'

        # Convert date format
        train_date = datetime.strptime(train_date_raw, "%d %b %Y").strftime("%d-%m-%Y")

        if train_code and train_date:
            train_codes.append((train_code, train_date))

    return train_codes


# Fetch current train codes
train_codes = get_train_codes()
print("Current Train Codes:", train_codes)

Current Train Codes: [('B580', '31-03-2025'), ('E848', '31-03-2025'), ('P423', '31-03-2025'), ('D839', '31-03-2025'), ('D332', '31-03-2025'), ('A876', '31-03-2025'), ('A810', '31-03-2025'), ('D540', '31-03-2025'), ('A314', '31-03-2025'), ('A230', '31-03-2025'), ('A450', '31-03-2025'), ('A528', '31-03-2025'), ('P229', '31-03-2025'), ('A231', '31-03-2025'), ('A531', '31-03-2025'), ('E260', '31-03-2025'), ('E950', '31-03-2025'), ('A731', '31-03-2025'), ('E132', '31-03-2025'), ('E847', '31-03-2025'), ('A730', '31-03-2025'), ('D225', '31-03-2025'), ('P227', '31-03-2025'), ('E261', '31-03-2025'), ('E949', '31-03-2025'), ('D425', '31-03-2025'), ('E262', '31-03-2025'), ('D938', '31-03-2025'), ('E846', '31-03-2025'), ('P670', '31-03-2025'), ('E263', '31-03-2025'), ('E133', '31-03-2025'), ('P714', '31-03-2025'), ('D837', '31-03-2025'), ('A131', '31-03-2025'), ('A130', '31-03-2025'), ('E948', '31-03-2025'), ('A914', '31-03-2025')]


In [40]:
import requests
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta
import pandas as pd

# API Namespace
NAMESPACE = {"ns": "http://api.irishrail.ie/realtime/"}

# Function to fetch train movements for a given TrainId and date
def get_train_movements(train_id, train_date):
    formatted_date = train_date.strftime("%d %b %Y")  # Convert to API format
    url = f"http://api.irishrail.ie/realtime/realtime.asmx/getTrainMovementsXML?TrainId={train_id}&TrainDate={formatted_date}"
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Failed to fetch data for {train_id} on {formatted_date}")
        return []
    
    root = ET.fromstring(response.text)
    movements = []
    
    for movement in root.findall("ns:objTrainMovements", NAMESPACE):
        movements.append({
            "TrainCode": train_id,
            "TrainDate": formatted_date,
            "LocationCode": movement.find("ns:LocationCode", NAMESPACE).text,
            "LocationFullName": movement.find("ns:LocationFullName", NAMESPACE).text,
            "ScheduledArrival": movement.find("ns:ScheduledArrival", NAMESPACE).text,
            "ActualArrival": movement.find("ns:Arrival", NAMESPACE).text,
            "ScheduledDeparture": movement.find("ns:ScheduledDeparture", NAMESPACE).text,
            "ActualDeparture": movement.find("ns:Departure", NAMESPACE).text,
            "LocationType": movement.find("ns:LocationType", NAMESPACE).text,
            "TrainOrigin": movement.find("ns:TrainOrigin", NAMESPACE).text,
            "TrainDestination": movement.find("ns:TrainDestination", NAMESPACE).text
        })
    
    return movements

# Function to collect historical data for past N days
def collect_historical_data(train_ids, days_back=30):
    all_data = []
    
    for days_ago in range(days_back):
        date_to_check = datetime.today() - timedelta(days=days_ago)
        print(f"Checking trains for {date_to_check.strftime('%d-%m-%Y')}...")
        
        for train_id in train_ids:
            train_data = get_train_movements(train_id, date_to_check)
            all_data.extend(train_data)
    
    return pd.DataFrame(all_data)

# Fetch current train IDs
train_codes = get_train_codes()
train_ids = [train[0] for train in train_codes]  # Extract Train IDs only

# Collect historical data
df = collect_historical_data(train_ids, days_back=3)

df.to_csv("irish_rail_historical_data.csv", index=False)
print(df.head())

Checking trains for 31-03-2025...
Checking trains for 30-03-2025...
Checking trains for 29-03-2025...
  TrainCode    TrainDate LocationCode    LocationFullName ScheduledArrival  \
0      P540  31 Mar 2025         CORK                Cork         00:00:00   
1      P540  31 Mar 2025        CK789                None         23:02:30   
2      P540  31 Mar 2025        RP805               RP805         23:11:30   
3      P540  31 Mar 2025        MW807               MW807         23:17:00   
4      P540  31 Mar 2025        KRLYJ  Killarney Junction         23:22:00   

  ActualArrival ScheduledDeparture ActualDeparture LocationType TrainOrigin  \
0          None           23:00:00            None            O        Cork   
1          None           23:02:30            None            T        Cork   
2          None           23:11:30            None            T        Cork   
3          None           23:17:00            None            T        Cork   
4          None           23:22:00

In [46]:
rows = df.loc[df['TrainDate'] == "29 Mar 2025"]
rows.head(50)

,TrainCode,TrainDate,LocationCode,LocationFullName,ScheduledArrival,ActualArrival,ScheduledDeparture,ActualDeparture,LocationType,TrainOrigin,TrainDestination
1029,A730,29 Mar 2025,HSTON,Dublin Heuston,00:00:00,None,20:30:00,20:30:00,O,Dublin Heuston,Galway
1030,A730,29 Mar 2025,IBJCT,Islandbridge Junction,20:32:00,None,20:32:00,None,T,Dublin Heuston,Galway
1031,A730,29 Mar 2025,ICADV,Inchicore Advance Starter,20:34:30,20:26:00,20:34:30,20:34:54,T,Dublin Heuston,Galway
1032,A730,29 Mar 2025,HK101,HK101,20:35:00,None,20:35:00,None,T,Dublin Heuston,Galway
1033,A730,29 Mar 2025,PWESF,Park West and Cherry Orchard,20:36:00,20:36:42,20:36:00,20:37:06,T,Dublin Heuston,Galway
1034,A730,29 Mar 2025,CLONF,Clondalkin,20:37:00,20:37:24,20:37:00,20:38:00,T,Dublin Heuston,Galway
1035,A730,29 Mar 2025,KISHF,Kishoge,20:37:30,20:38:30,20:37:30,20:38:42,T,Dublin Heuston,Galway
1036,A730,29 Mar 2025,ADAMF,Adamstown,20:39:00,20:39:42,20:39:00,20:39:54,T,Dublin Heuston,Galway
1037,A730,29 Mar 2025,HAZEF,Hazelhatch,20:40:30,20:41:24,20:40:30,20:41:30,T,Dublin Heuston,Galway
1038,A730,29 Mar 2025,HK151,HK151,20:41:00,None,20:41:00,None,T,Dublin Heuston,Galway
